## Crypto Sentiment Analyzer

In [1]:
# Imports

## Establish Baseline Performance

#### Step 1: Import the OHLCV dataset into a Pandas DataFrame.

In [3]:
# Import the OHLCV dataset into a 

# Review the DataFrame

In [4]:

# Filter the date index and close columns

# Use the pct_change function to generate returns from close prices

# Drop all NaN values from the DataFrame

# Review the DataFrame


#### Step 2: Generate trading signals using short- and long-window SMA values.

 #### Step 3: Split the data into training and testing datasets.

#### Step 4: Use the SVC classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

#### Step 5: Review the classification report associated with the SVC model predictions.

#### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

#### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.



#### Tune the Baseline Trading Algorithm


## Evaluate a New Machine Learning Classifier

#### Step 1: Import a new classifier, such as AdaBoost, DecisionTreeClassifier, or LogisticRegression. (For the full list of classifiers, refer to the Supervised learning page in the scikit-learn documentation.)

#### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

#### Step 3: Backtest the new model to evaluate its performance.